# Creating a quantum simulator

This notebook contains a quantum simulator built from scratch using basic libararies. The simulator performs the following functions:

*   Initialize a state
*   Run a program by calculating for each gate, a matrix operator and then applying it to the qubits
*   Perform multi-shot measurements of all qubits using weighted random technique


# Importing basic libraries

In [1]:
import numpy as np
import json
from collections import Counter

# Defining Quantum Gates

Our simulator will support the following gates:

*   Identity Gate
*   Hadamard Gate
*   Pauli-X Gate
*   Pauli-Y Gate
*   Pauli-Z Gate
*   S Gate
*   T Gate
*   CNOT

Definitions for each gate can be [found here](https://qiskit.org/textbook/ch-states/single-qubit-gates.html).

In [2]:
# A dictionary containing the standard quantum gates 

gates = {
    "i":np.identity(2),
    "h":np.array([[1/np.sqrt(2), 1/np.sqrt(2)],[1/np.sqrt(2), -1/np.sqrt(2)]]),
    "x":np.array([[0, 1],[1, 0]]),
    "y":np.array([[0, +1j],[-1j, 0]]),
    "z":np.array([[1, 0],[0, -1]]),
    "s":np.array([[1, 0],[0, np.exp(np.pi * +1j / 2)]]),
    "t":np.array([[1, 0],[0, np.exp(np.pi * +1j / 4)]]),
}

# Defining the quantum simulator

Since qubits can exist in more states than just the traditional 0 and 1 of a bit, we can encode more information using them. In practice, it is possible to encode a classical bit using a qubit, but the converse is not true. You cannot encode a qubit of information in a classical transistor. Using bits, an n-component system can be encoded using n transistors; only 8 stored bits are required to encapsulate an 8-bit classical system. If the n-component system were instead to be quantum, 2ⁿ complex numbers would be required to encode it (Kopczyk, 2018). By extension, encoding an 8-qubit quantum computer requires 256 complex numbers. And to simulate 64 qubits one will need 2⁶⁴=18, 446, 744, 073, 709, 551, 616 complex numbers on a classical machine.

- From [The Gradient](https://thegradient.pub/knocking-on-turings-door-quantum-computing-and-machine-learning/).

In [3]:
def get_ground_state(num_qubits):
    """
    Takes the number of qubits and returns a vector in the ground state representing |0>
    @param num_qubits: number of qubits in the system 
    @return: vector of 2^n, where n is num_qubits, representing state |0> 
    """
    vector = [0 for i in range(2**num_qubits)]
    vector[0] = 1
    return np.array(vector)

def get_single_qubit_operator(total_qubits, gate, target):
    """
    Takes the number of qubits and returns a matrix operator of size 2^n x 2^n, where n is total_qubits, for required gate
    @param total_qubits: count of qubits in the system
    @param gate: the input gate for which the operator is needed
    @param target: the target qubit(s) on which the gate is applied in the representation defined in [Link: shorturl.at/lpqxQ]
    @return: matrix operator of size 2^n x 2^n, where n is total_qubits
    """
    #starting tensor product with identity or the gate itself
    operator = gate if target == 0 else gates['i']

    for qubit in range(1, total_qubits):
        if qubit == target:
            #tensor product with the gate
            operator = np.kron(operator, gate)
        else:
            #tensor product with identity
            operator = np.kron(operator, gates['i'])
    
    return operator

def get_cx_operator(total_qubits, control, target):
    """
    Takes the number of qubits and returns a matrix operator of CNOT of size 2^n x 2^n, where n is total_qubits
    @param total_qubits: count of qubits in the system
    @param gate: the input gate for which the operator is needed; CNOT in this case
    @param target: the target qubit(s) in the representation defined in [Link: shorturl.at/lpqxQ]
    @return: CNOT matrix operator of size 2^n x 2^n, where n is total_qubits
    """
    P0x0 = np.array([[1, 0],[0, 0]])
    P1x1 = np.array([[0, 0],[0, 1]])
    X = gates['x']

    # case 1
    operator1 = P0x0 if control == 0 else gates['i']

    for qubit in range(1, total_qubits):
        if qubit == control:
            operator1 = np.kron(operator1, P0x0)
        else:
            operator1 = np.kron(operator1, gates['i'])

    # case 2
    operator2 = P1x1 if control == 0 else gates['i']

    for qubit in range(1, total_qubits):
        if qubit == control:
            operator2 = np.kron(operator2, P1x1)
        elif qubit == target:
            operator2 = np.kron(operator2, X)
        else:
            operator2 = np.kron(operator2, gates['i'])
    
    #take the sum of both cases to generate the final matrix operator
    return operator1 + operator2

def get_operator(total_qubits, gate_unitary, target_qubits):
    """
    Wrapper function for generating matrix operator to handle CNOT and all other gates separately
    @param total_qubits: count of qubits in the system
    @param gate_unitary: the input gate for which the operator is needed
    @param target_qubits: the qubit(s) onto which the gate is to be applied
    @return: matrix operator of size 2^n x 2^n, where n is total_qubits
    """
    # return unitary operator of size 2**n x 2**n for given gate and target qubits

    if gate_unitary == 'cx':
        return get_cx_operator(total_qubits, target_qubits[0], target_qubits[1])
    else:
        return get_single_qubit_operator(total_qubits, gates[gate_unitary] , target_qubits[0])


def run_program(initial_state, program):
    """
    Main function that builds and runs the quantum circuit
    @param initial_state: a state vector of n-qubits as input to the quantum system  
    @param program: an array of objects, each containing "gate" and "target" for defining the gate operation and target qubits for it 
    @return: final/evolved state of the quantum system after running the program
    """
    total_qubits = int(np.log2(len(initial_state)))

    state = initial_state
    
    #for each instruction:
    #   get matrix operator
    #   apply it to state
    #   move ahead
    for instruction in program: 
        gate = instruction['gate']
        targets = instruction['target']
        operator = get_operator(total_qubits, gate, targets)
        state = np.dot(operator, state)

    return state

def measure_all(state_vector):
    """
    Measures qubits by taking them from |+> and |-> basis to the computational basis 
    @param state_vector: a state vector of n-qubits that are to be measured  
    @return: index (or position) of the quantum state measured
    """
    # choose element from state_vector using weighted random and return it's index
    probabilities = np.abs(state_vector)**2
    index = np.random.choice(a=len(state_vector), p=probabilities)

    return index

def get_counts(state_vector, num_shots):
    """
    Executes quantum circuit num_shots times and return the probability distribution of each output through a dictionary 
    @param state_vector: a state vector of n-qubits
    @param num_shots: number of times that the program is executes  
    @return: dictionary containing the output state and its frequency
    """
    # simply execute measure_all in a loop num_shots times and
    # return object with statistics in following form:
    #   {
    #      element_index: number_of_ocurrences,
    #      element_index: number_of_ocurrences,
    #      element_index: number_of_ocurrences,
    #      ...
    #   }
    # (only for elements which occoured - returned from measure_all)

    results = []
    
    num_bits = int(np.log2(len(state_vector)))
    

    for _ in range(num_shots):
        result = measure_all(state_vector)
        results.append("{0:b}".format(result).zfill(num_bits)[::-1]) 

    stats = Counter(results)   

    return json.dumps(stats, sort_keys=True, indent=4)

# A sample program to execute using the simulator

In [4]:
# Define program:

my_circuit = [
{ "gate": "h", "target": [0] }, 
{ "gate": "cx", "target": [0, 1] }
]


# Create "quantum computer" with 2 qubits (this is actually just a vector :) )
my_qpu = get_ground_state(3)

# Run circuit
final_state = run_program(my_qpu, my_circuit)

# Read results
counts = get_counts(final_state, 1000)
print(counts)

# Should print something like:
# {
#   "00": 502,
#   "11": 498
# }

# Voila!

{
    "000": 500,
    "011": 500
}
